# Check funding rate entities

* Query positions to ensure `netFunding` is recorded properly
* Check `fundingRates` value on the entities for historical funding rate chart


In [1]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.express as px

load_dotenv()


True

In [2]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# mainnet
SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/tburm/optimism-perps'
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# testnet
# SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-goerli-perps'
# RPC_ENDPOINT = f'https://optimism-kovan.infura.io/v3/{INFURA_KEY}'


In [3]:
# functions
convertDecimals = lambda x: Decimal(x) / Decimal(10**18)
convertBytes = lambda x: bytearray.fromhex(x[2:]).decode().replace('\x00', '')

def clean_df(df, decimal_cols=[], bytes_cols=[]):
    for col in decimal_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertDecimals)
        else:
            print(f"{col} not in DataFrame")
    for col in bytes_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertBytes)
        else:
            print(f"{col} not in DataFrame")
    return df

async def run_query(query, params, endpoint=SUBGRAPH_ENDPOINT):
    transport = AIOHTTPTransport(url=endpoint)

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        query = query

        result = await session.execute(query, variable_values=params)
        df = pd.DataFrame(result)
        return df

async def run_recursive_query(query, params, accessor, endpoint=SUBGRAPH_ENDPOINT):
  transport = AIOHTTPTransport(url=endpoint)

  async with Client(
      transport=transport,
      fetch_schema_from_transport=True,
  ) as session:
    done_fetching = False
    all_results = []
    while not done_fetching:
      result = await session.execute(query, variable_values=params)
      if len(result[accessor]) > 0:
        all_results.extend(result[accessor])
        params['last_id'] = all_results[-1]['id']
      else:
        done_fetching = True
    
    df = pd.DataFrame(all_results)
    return df


In [4]:
position_query = gql("""
query positions(
    $last_id: ID!
  ) {
    futuresPositions(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      account
      asset
      initialMargin
      size
      entryPrice
      avgEntryPrice
      lastPrice
      exitPrice
      pnl
      pnlWithFeesPaid
      openTimestamp
      closeTimestamp
      feesPaid
      netFunding
      isLiquidated
      isOpen
      margin
      netTransfers
      totalDeposits
      totalVolume
    }  
}
""")

funding_query = gql("""
query funding(
    $last_id: ID!
  ) {
    fundingRateUpdates(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      timestamp
      market
      marketKey
      asset
      funding
      fundingRate
      sequenceLength
    }  
}
""")


### Run a query

In [5]:
position_params = {
    'last_id': ''
}

position_cols = [
    'avgEntryPrice',
    'entryPrice',
    'exitPrice',
    'feesPaid',
    'initialMargin',
    'lastPrice',
    'margin',
    'netFunding',
    'netTransfers',
    'pnl',
    'pnlWithFeesPaid',
    'size',
    'totalDeposits',
    'totalVolume'
]

df_position = await run_recursive_query(position_query, position_params, 'futuresPositions')
df_position = df_position.replace({None: np.nan})
df_position = clean_df(df_position, decimal_cols=position_cols)


In [6]:
df_position

,id,account,asset,initialMargin,size,entryPrice,avgEntryPrice,lastPrice,exitPrice,pnl,...,openTimestamp,closeTimestamp,feesPaid,netFunding,isLiquidated,isOpen,margin,netTransfers,totalDeposits,totalVolume
0,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x0,0x8feafd5e0fcefdd2624906c0f913d563306aae17,0x4d415449430000000000000000000000000000000000...,77.199529739466510105,0,1.336,1.336,1.344387130045637299,NaN,0,...,1675952475,NaN,0,0,False,True,2998,19477.291298495857635199,559754.695995731883463706,0
1,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x1,0x8feafd5e0fcefdd2624906c0f913d563306aae17,0x4d415449430000000000000000000000000000000000...,75.199529739466510105,0,1.340312948674674971,1.340312948674674971,1.341738286882998216,1.341738286882998216,2.052092628903229758,...,1675952535,1675952960,7.861001237100389934,-8.08234215782E-7,False,False,71.390620323035134147,0,0,3861.411655626170581108
2,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x10,0xc8728ae130381eb77fc9a8b715564b00e83e19df,0x4d415449430000000000000000000000000000000000...,197.498040648137408231,0,1.252917520139616653,1.252917520139616653,1.215543235997026069,1.215543235997026069,29.769574101247450474,...,1676067093,1676274397,5.57295374788655442,5.275340257241933773,False,False,228.970001258740238059,0,0,1966.192184858193026125
3,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x11,0xad81c90131f4a10f5d324164f4406dc43c962d37,0x4d415449430000000000000000000000000000000000...,398,0,1.244031360015537303,1.244031360015537303,1.251415209660742221,1.251415209660742221,47.509613880688074696,...,1676068494,1676178069,20.046851968746984601,-20.186115562847729165,False,False,407.27664634909336093,0,0,16056.353891523122217302
4,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x12,0x5e7270058456bfcbde96da2240672b6c526cc06b,0x4d415449430000000000000000000000000000000000...,548,0,1.245751535877516588,1.245751535877516588,1.169319730586746629,1.169319730586746629,168.679621120817358384,...,1676068825,1676314293,9.29615658135243301,15.343425345830345921,False,False,724.726889885295271296,0,0,5329.892505576596482213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5541,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xb,0x78f214cf1287b5c97df55a285e885b09c0d43e3d,0x58414700000000000000000000000000000000000000...,162.739186100680223994,0,21.695059131724097826,21.695059131724097826,21.635837322225261266,21.635837322225261266,-15.956808702549099059,...,1676406488,1676542759,7.75206860911845269,-0.091856510899806127,False,False,250.790121166102415797,109.851668887989549739,171.094760703526629418,11675.138457888834906538
5542,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xc,0x88747e96c045e71b357ce0b75597492beaa1fb8f,0x58414700000000000000000000000000000000000000...,98,0,21.728833842533239428,21.728833842533239428,21.580583270495936189,21.580583270495936189,-13.642729241618423108,...,1676575702,1676587925,4.598513203653544012,-0.007461465738755152,False,False,81.751296088989277728,0,0,3985.540447943087280318
5543,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xd,0x11ba886f23173da913aa3dbc51492e8080a781ac,0x58414700000000000000000000000000000000000000...,4998,0,21.482620917757409705,21.482620917757409705,21.82713009230149882,21.82713009230149882,641.506744317628356061,...,1676606928,1676883214,16.064916305069556019,-8.521098358010335535,False,False,5616.920729654548464506,0,0,80646.610948569582916407
5544,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xe,0x41dbd3a603adfb85c47fee7cf999d32bcc36a4f1,0x58414700000000000000000000000000000000000000...,183.427032240973901673,0,21.728166360389986619,21.728166360389986619,21.816099329108998862,21.816099329108998862,14.574525202046228121,...,1676864380,1677033107,5.081863514639043735,-0.794828372462505745,False,False,194.124865555918580315,0,0,7217.281605994306985757


In [7]:
df_position.loc[df_position['netFunding'].abs() > 0, :]

,id,account,asset,initialMargin,size,entryPrice,avgEntryPrice,lastPrice,exitPrice,pnl,...,openTimestamp,closeTimestamp,feesPaid,netFunding,isLiquidated,isOpen,margin,netTransfers,totalDeposits,totalVolume
1,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x1,0x8feafd5e0fcefdd2624906c0f913d563306aae17,0x4d415449430000000000000000000000000000000000...,75.199529739466510105,0,1.340312948674674971,1.340312948674674971,1.341738286882998216,1.341738286882998216,2.052092628903229758,...,1675952535,1675952960,7.861001237100389934,-8.08234215782E-7,False,False,71.390620323035134147,0,0,3861.411655626170581108
2,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x10,0xc8728ae130381eb77fc9a8b715564b00e83e19df,0x4d415449430000000000000000000000000000000000...,197.498040648137408231,0,1.252917520139616653,1.252917520139616653,1.215543235997026069,1.215543235997026069,29.769574101247450474,...,1676067093,1676274397,5.57295374788655442,5.275340257241933773,False,False,228.970001258740238059,0,0,1966.192184858193026125
3,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x11,0xad81c90131f4a10f5d324164f4406dc43c962d37,0x4d415449430000000000000000000000000000000000...,398,0,1.244031360015537303,1.244031360015537303,1.251415209660742221,1.251415209660742221,47.509613880688074696,...,1676068494,1676178069,20.046851968746984601,-20.186115562847729165,False,False,407.27664634909336093,0,0,16056.353891523122217302
4,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x12,0x5e7270058456bfcbde96da2240672b6c526cc06b,0x4d415449430000000000000000000000000000000000...,548,0,1.245751535877516588,1.245751535877516588,1.169319730586746629,1.169319730586746629,168.679621120817358384,...,1676068825,1676314293,9.29615658135243301,15.343425345830345921,False,False,724.726889885295271296,0,0,5329.892505576596482213
5,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-0x13,0x0818f6e98a6b62e02a6a258458dfb579fd2387c6,0x4d415449430000000000000000000000000000000000...,2998,0,1.23645936237868536,1.23645936237868536,1.229974046348866969,1.229974046348866969,-62.918430987714750432,...,1676070645,1676073394,27.941120013387782681,-0.540968819361581545,False,False,2908.599480179535885343,0,0,23928.536327190239732471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5540,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xa,0x41dbd3a603adfb85c47fee7cf999d32bcc36a4f1,0x58414700000000000000000000000000000000000000...,72.187156284777411218,0,21.85831402623644432,21.85831402623644432,21.629594960081070536,21.629594960081070536,-19.406120610567139209,...,1676400978,1676490874,4.554442927099562886,-0.013924684118524924,False,False,670.639309529243999754,621.010811677606825675,637.010811677606825675,3689.817473793563533457
5541,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xb,0x78f214cf1287b5c97df55a285e885b09c0d43e3d,0x58414700000000000000000000000000000000000000...,162.739186100680223994,0,21.695059131724097826,21.695059131724097826,21.635837322225261266,21.635837322225261266,-15.956808702549099059,...,1676406488,1676542759,7.75206860911845269,-0.091856510899806127,False,False,250.790121166102415797,109.851668887989549739,171.094760703526629418,11675.138457888834906538
5542,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xc,0x88747e96c045e71b357ce0b75597492beaa1fb8f,0x58414700000000000000000000000000000000000000...,98,0,21.728833842533239428,21.728833842533239428,21.580583270495936189,21.580583270495936189,-13.642729241618423108,...,1676575702,1676587925,4.598513203653544012,-0.007461465738755152,False,False,81.751296088989277728,0,0,3985.540447943087280318
5543,0xdcb8438c979fa030581314e5a5df42bbfed744a0-0xd,0x11ba886f23173da913aa3dbc51492e8080a781ac,0x58414700000000000000000000000000000000000000...,4998,0,21.482620917757409705,21.482620917757409705,21.82713009230149882,21.82713009230149882,641.506744317628356061,...,1676606928,1676883214,16.064916305069556019,-8.521098358010335535,False,False,5616.920729654548464506,0,0,80646.610948569582916407


In [8]:
df_position['pnl'].sum() + df_position['netFunding'].sum() - df_position['feesPaid'].sum()

Decimal('-137165.636783540719142438')

In [9]:
df_position['feesPaid'].sum()

Decimal('420691.509585595046159041')

In [10]:
df_position['pnlWithFeesPaid'].sum()



Decimal('-127642.636783540719142438')

## Historical funding rate

In [11]:
funding_params = {
    'last_id': ''
}

funding_decimal_cols = [
    'funding',
    'fundingRate'
]

funding_bytes_cols = [
    'marketKey',
    'asset'
]

df_funding = await run_recursive_query(funding_query, funding_params, 'fundingRateUpdates')
df_funding = df_funding.replace({None: np.nan})
df_funding = clean_df(df_funding, decimal_cols=funding_decimal_cols, bytes_cols=funding_bytes_cols)


In [12]:
df_funding['timestamp'] = df_funding['timestamp'].astype(np.int64)
df_funding['fundingRate'] = df_funding['fundingRate'].astype(np.float64)


In [13]:
df_funding

,id,timestamp,market,marketKey,asset,funding,fundingRate,sequenceLength
0,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-1,1675952475,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042,sMATICPERP,MATIC,0,0.000000e+00,1
1,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-10,1675960087,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042,sMATICPERP,MATIC,-1.9047554247E-8,1.699673e-07,10
2,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-100,1676082221,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042,sMATICPERP,MATIC,-0.001322592712477598,1.551538e-03,100
3,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-1000,1676748461,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042,sMATICPERP,MATIC,-0.023724414029589254,5.880351e-04,1000
4,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042-1001,1676750878,0x074b8f19fc91d6b2eb51143e1f186ca0ddb88042,sMATICPERP,MATIC,-0.023747721570361566,5.330023e-04,1001
...,...,...,...,...,...,...,...,...
36253,0xdcb8438c979fa030581314e5a5df42bbfed744a0-95,1677033107,0xdcb8438c979fa030581314e5a5df42bbfed744a0,sXAGPERP,XAG,-0.009786980809347932,1.269254e-04,95
36254,0xdcb8438c979fa030581314e5a5df42bbfed744a0-96,1677033152,0xdcb8438c979fa030581314e5a5df42bbfed744a0,sXAGPERP,XAG,-0.009788424095941762,1.269323e-04,96
36255,0xdcb8438c979fa030581314e5a5df42bbfed744a0-97,1677038477,0xdcb8438c979fa030581314e5a5df42bbfed744a0,sXAGPERP,XAG,-0.009959739826614255,1.277435e-04,97
36256,0xdcb8438c979fa030581314e5a5df42bbfed744a0-98,1677177995,0xdcb8438c979fa030581314e5a5df42bbfed744a0,sXAGPERP,XAG,-0.014718547604870155,1.489971e-04,98


In [14]:
df_funding_sorted = df_funding.sort_values(['asset', 'timestamp'])


In [15]:
fig = px.line(
    df_funding_sorted,
    x='timestamp',
    y='fundingRate',
    color='asset'
)

fig.show()